# EDA notebook

## 1. Load the dataset
Inside the notebook, fetch the artifact sample.csv from W&B and read it into pandas

In [1]:
import wandb
import pandas as pd

run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

/home/u339700/miniconda3/envs/mlflow-6f02ff4842fbe0d4c84a3c2da9d0eede237ec11e/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/u339700/miniconda3/envs/mlflow-6f02ff4842fbe0d4c84a3c2da9d0eede237ec11e/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to

## 2. Generate the profile report
Use ydata-profiling (the modern replacement for the deprecated pandas-profiling) to generate an EDA report

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20000 non-null  int64  
 1   name                            19993 non-null  object 
 2   host_id                         20000 non-null  int64  
 3   host_name                       19992 non-null  object 
 4   neighbourhood_group             20000 non-null  object 
 5   neighbourhood                   20000 non-null  object 
 6   latitude                        20000 non-null  float64
 7   longitude                       20000 non-null  float64
 8   room_type                       20000 non-null  object 
 9   price                           20000 non-null  int64  
 10  minimum_nights                  20000 non-null  int64  
 11  number_of_reviews               20000 non-null  int64  
 12  last_review                     

In [2]:
import ydata_profiling

profile = ydata_profiling.ProfileReport(df)
profile.to_widgets()

ModuleNotFoundError: No module named 'pkg_resources'

In [10]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
count,1.900100e+04,1.900100e+04,19001.000000,19001.000000,19001.000000,19001.000000,19001.000000,15243,15243.000000,19001.000000,19001.000000
mean,1.883041e+07,6.639459e+07,40.728063,-73.950827,122.340456,6.906900,23.797747,2018-10-01 08:24:05.384766720,1.380928,6.583811,109.725383
min,2.539000e+03,2.571000e+03,40.508730,-74.239140,10.000000,1.000000,0.000000,2011-05-12 00:00:00,0.010000,1.000000,0.000000
25%,9.355498e+06,7.728754e+06,40.688820,-73.982050,66.000000,1.000000,1.000000,2018-06-29 12:00:00,0.190000,1.000000,0.000000
50%,1.938754e+07,3.048785e+07,40.721710,-73.954630,100.000000,2.000000,6.000000,2019-05-19 00:00:00,0.720000,1.000000,39.000000
75%,2.891952e+07,1.048354e+08,40.763210,-73.934490,160.000000,5.000000,24.000000,2019-06-23 00:00:00,2.010000,2.000000,219.000000
max,3.648561e+07,2.742733e+08,40.913060,-73.717950,350.000000,1250.000000,607.000000,2019-07-08 00:00:00,27.950000,327.000000,365.000000
std,1.096986e+07,7.782663e+07,0.055389,0.046825,71.530346,21.456544,45.493455,NaN,1.689988,31.154750,130.599899


## 3. Apply basic fixes
We now apply two important fixes:
- Remove price outliers
- Convert last_review to datetime

In [5]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()

# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

## 4. Verify the fixes
Either regenerate the profile report or simply inspect the DataFrame
Confirm that:
- Prices are within the defined range ✅
- last_review is now of datetime type ✅

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  int64         
 11  number_

In [7]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,9138664,Private Lg Room 15 min to Manhattan,47594947,Iris,Queens,Sunnyside,40.74271,-73.92493,Private room,74,2,6,2019-05-26,0.13,1,5
1,31444015,TIME SQUARE CHARMING ONE BED IN HELL'S KITCHEN...,8523790,Johlex,Manhattan,Hell's Kitchen,40.76682,-73.98878,Entire home/apt,170,3,0,NaT,NaN,1,188
2,8741020,Voted #1 Location Quintessential 1BR W Village...,45854238,John,Manhattan,West Village,40.73631,-74.00611,Entire home/apt,245,3,51,2018-09-19,1.12,1,0
3,34602077,Spacious 1 bedroom apartment 15min from Manhattan,261055465,Regan,Queens,Astoria,40.76424,-73.92351,Entire home/apt,125,3,1,2019-05-24,0.65,1,13
4,23203149,Big beautiful bedroom in huge Bushwick apartment,143460,Megan,Brooklyn,Bushwick,40.69839,-73.92044,Private room,65,2,8,2019-06-23,0.52,2,8


In [11]:
df['price'].describe()

count    19001.000000
mean       122.340456
std         71.530346
min         10.000000
25%         66.000000
50%        100.000000
75%        160.000000
max        350.000000
Name: price, dtype: float64

In [ ]:
run.finish()